In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%set_env PYTHONBREAKPOINT=IPython.core.debugger.set_trace

env: PYTHONBREAKPOINT=IPython.core.debugger.set_trace


In [2]:
from fastai.vision import *

In [3]:
vcand = np.load('vcand_case2.npy')
nview = 20
viewOrd = []

In [4]:
path = Path('./ModelNet40_20')
path.ls()

[PosixPath('ModelNet40_20/val'),
 PosixPath('ModelNet40_20/test'),
 PosixPath('ModelNet40_20/train'),
 PosixPath('ModelNet40_20/item_list.txt'),
 PosixPath('ModelNet40_20/.ipynb_checkpoints'),
 PosixPath('ModelNet40_20/models')]

In [5]:
classes = sorted(os.listdir(path/'train'))

In [6]:
# For the validation dataset, count the number of data points for each class.
# Used to calculate accuracy-per-class during validation.

def directory(path,extension):
    list_dir = []
    list_dir = os.listdir(path)
    count = 0
    for file in list_dir:
        if file.endswith(extension): # eg: '.txt'
            count += 1
    return count

folders = [f.path for f in os.scandir(path/'test') if f.is_dir()]
folders = sorted(folders)
validCount = []
for folds in folders:
    validCount = np.append(validCount, directory(folds, '.png')//20)
validCount = validCount.astype(int)

In [7]:
# u = np.arange(0, 40)
# counts = np.random.randint(low=0, high = 100, size = 40)
# v = [validCount[i] for i in u.astype(int)]
# unique = [classes[i] for i in u.astype(int)]
# corrects = ((counts/v * 100) + 0.5).astype(int) / 100.0
# countDict = dict(zip(unique, corrects))
# countDict

#         u, counts = np.unique(self.incTargs, return_counts=True)
#         v = [validCount[i] for i in u.astype(int)]
#         u = [classes[i] for i in u.astype(int)]
#         countDict = dict(zip(u, (counts, v)))

In [8]:
tfms = get_transforms(do_flip=False, flip_vert=False, max_rotate=0.0,
                      max_lighting=0.0, max_zoom=1.05, max_warp=0.0)

In [9]:
src = (ImageList.from_folder(path)
      .split_by_folder(train='train', valid='test')
      .label_from_folder())
src

LabelLists;

Train: LabelList (196860 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
mantel,mantel,mantel,mantel,mantel
Path: ModelNet40_20;

Valid: LabelList (49360 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
mantel,mantel,mantel,mantel,mantel
Path: ModelNet40_20;

Test: None

In [10]:
data = (src.transform(tfms, size=128)
        .databunch(bs=400).normalize(imagenet_stats))

data.train_dl = data.train_dl.new(shuffle=False)
data.valid_dl = data.valid_dl.new(shuffle=False)

if data.train_dl.dl.batch_size % nview != 0:
    print ('Error: batch size must be a multiplication of the number of views,', nview)
    exit()
    
sorted_img = sorted(data.train_ds.x.items)
sorted_cats = sorted(data.train_ds.y.items)

In [11]:
# random permutation for validation set. Probably not necessary, but just an experiment to see if it makes
# any difference during training/validation.
sorted_vimg = sorted(data.valid_ds.x.items)
sorted_vcats = sorted(data.valid_ds.y.items)

val_nsamp = int( len(data.valid_ds) / nview )

inds = np.zeros( ( nview, val_nsamp ) ).astype('int')
inds[ 0 ] = np.random.permutation(range(val_nsamp)) * nview
for i in range(1,nview):
    inds[ i ] = inds[ 0 ] + i
inds = inds.T.reshape( nview * val_nsamp )
print(inds)

IL  = ImageList([sorted_vimg[i] for i in inds], path=path)
data.valid_ds.x = IL
CL = CategoryList([sorted_vcats[i] for i in inds], classes=classes, path=path)
data.valid_ds.y = CL
# data.valid_ds.x = ImageList(sorted(data.valid_ds.x.items), path=path)
# data.valid_ds.y = CategoryList(sorted(data.valid_ds.y.items), classes=classes, path=path)

[47820 47821 47822 47823 ...  1756  1757  1758  1759]


In [12]:
# idx = random.randint(0, 50000)
# print(data.valid_ds[idx][1], idx)
# data.valid_ds.items[idx]

In [13]:
data.valid_ds

LabelList (49360 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: CategoryList
vase,vase,vase,vase,vase
Path: ModelNet40_20

In [14]:
# idx = 8779
# print(data.train_ds[idx])
# data.train_ds[idx][0]

In [15]:
# class updateTargetRN(LearnerCallback):
#     _order=10
#     "Overwrites the target labels for the rotationNet implementation."
#     def on_batch_begin(self, last_input:Tensor, last_target:Tensor, train:bool=True, **kwargs)->None:
#         print('Entering updateTargetRN!')
        

In [16]:
# class FineTuneModel(nn.Module):
#     def __init__(self, original_model, arch, num_classes):
#         super(FineTuneModel, self).__init__()
#         self.features = original_model.features
#         self.classifier = nn.Sequential(
#             nn.Dropout(),
#             nn.Linear(256 * 6 * 6, 4096),
#             nn.ReLU(inplace=True),
#             nn.Dropout(),
#             nn.Linear(4096, 4096),
#             nn.ReLU(inplace=True),
#             nn.Linear(4096, num_classes),
#         )
        
#     # Freeze those weights
# #     for p in self.features.parameters():
# #         p.requires_grad = False
        
#     def forward(self, x):
#         f = self.features(x)
#         f = f.view(f.size(0), 256 * 6 * 6)
#         y = self.classifier(f)
#         return y

In [17]:
# model = models.resnet18(pretrained=True)
# model.avgpool = nn.AdaptiveAvgPool2d(1)
# model = FineTuneModel(model, 'resnet', numClasses )
# model.features

In [18]:
class GenerateTargetrN(LearnerCallback):

    "Creates the target labels for a rotationNet implementation."
    
    def __init__(self, learn:Learner):
        super().__init__(learn)
        self.target_var = torch.LongTensor( learn.data.train_dl.dl.batch_size * nview )
        self.loss_func = learn.loss_func
        self.output = torch.Tensor()
        
#     def on_batch_begin(self, last_input, last_target, **kwargs)->None:
#         print ('Entering on_batch_begin!')
#         img = last_input[0].clone().cpu()
#         img = img.numpy()
#         img = img.transpose(1,2,0)
#         img = abs(img*85).astype(int)
#         imgplot = plt.imshow(img)
        
    def on_loss_begin(self, last_output, last_target, train, **kwargs:Any)->None:
        
        if not train:
            return {'last_output':last_output, 'last_target':last_target}

        nsamp = int( last_output.size(0) / nview )
#         print('Number of samples: ', nsamp)

#         input_var = torch.autograd.Variable(input)
        target_ = torch.LongTensor( last_target.size(0) * nview )
#         print('Last_target: ', last_target, last_target.shape, target_.shape)

        # compute output
        self.output = last_output
#         print('self.output initial shape: ', self.output.shape)
        num_classes = int( self.output.size( 1 ) / nview ) - 1
#         print('Number of classes: ', num_classes)
        self.output = self.output.view( -1, num_classes + 1 )
#         print('self.output after reshaping: ', self.output)

        # compute scores and decide target labels
        output_ = torch.nn.functional.log_softmax( self.output, dim=1 ) #try sigmoid instead?
#         print('output_ after sofmax: ', output_.shape)

        #subtracts the last col from every other col, removes last col
        output_ = output_[ :, :-1 ] - torch.t( output_[ :, -1 ].repeat(1, output_.size(1)-1).view(output_.size(1)-1, -1) )
#         print('output_ after subtracting last column', output_.shape)
        output_ = output_.view( -1, nview * nview, num_classes )
#         print('output_', output_)
        output_ = output_.data.cpu().numpy()

        '''Arrange as:
           [x,:,:] = all views activations for image[x//num views]
           [:,y,:] = view activations for class[y]
           [:,:,z] = sample set[z]'''

        output_ = output_.transpose( 1, 2, 0 )
#         print('output_: ', output_)

        #default view variable is the incorrect view
        for j in range(target_.size(0)):
            target_[ j ] = num_classes

        # Initialize scores to 0
        scores = np.zeros( ( vcand.shape[ 0 ], num_classes, nsamp ) )

        #add up scores for each of the candidates for viewpoint variables
        for j in range(vcand.shape[0]):
            for k in range(vcand.shape[1]):
                scores[ j ] = scores[ j ] + output_[ vcand[ j ][ k ] * nview + k ]

        for n in range( nsamp ):
            #finds max score for column that corresponds to target label (for each class label and sample group)
            j_max = np.argmax( scores[ :, last_target[ n * nview ], n ] )
            # Assign target labels. Only 1 view per image gets set to class label.
            # Others remain default, which is the incorrect view
            for k in range(vcand.shape[1]):
                target_[ n * nview * nview + vcand[ j_max ][ k ] * nview + k ] = last_target[ n * nview ]

#         print('Final targets: ', target_.shape)

        target_ = target_.cuda()
        self.target_var = torch.autograd.Variable(target_)
#         print('self.target_var: ', self.target_var)

#             print(list(learn.model.parameters())[-4][:5])

        return {'last_output':self.output, 'last_target':self.target_var}
    
#     def on_backward_begin(self, **kwargs):
# #         print("Entering on_backward_begin!")
# #         print('Last_loss: ', last_loss)
#         if not is_listy(self.target_var):
#             loss = self.loss_func(self.output, *[self.target_var])
#         else:
#             loss = self.loss_func(self.output, *self.target_var)
# #         print('Updated loss: ', loss)
#         return {'last_loss':loss}

In [19]:
class rNAccuracy(LearnerCallback):
    _order=-20
    def __init__(self, learn):
        super().__init__(learn)
  
    def on_train_begin(self, **kwargs):
        self.learn.recorder.add_metric_names(['rN_t_loss', 'Prec@1', 'Prec@5'])
        self.vcount_tot = 0
        self.prec1, self.prec5 = 0., 0.
        
    def on_epoch_begin(self, **kwargs):
        self.tloss_tot, self.vloss_tot, self.tcount, self.vcount= 0., 0., 0, 0
        self.incTargs = []
        self.viewOrd = []
        
        #print weights to confirm body is frozen or not
#         print(list(learn.model.parameters())[0][0][0][1])
        
        # random permutation
        train_nsamp = int( len(learn.data.train_ds) / nview )
        
        inds = np.zeros( ( nview, train_nsamp ) ).astype('int')
        inds[ 0 ] = np.random.permutation(range(train_nsamp)) * nview
        for i in range(1,nview):
            inds[ i ] = inds[ 0 ] + i
        inds = inds.T.reshape( nview * train_nsamp )
        
        IL  = ImageList([sorted_img[i] for i in inds], path=path)
        learn.data.train_ds.x = IL
        CL = CategoryList([sorted_cats[i] for i in inds], classes=classes, path=path)
        learn.data.train_ds.y = CL
        # Print a random data item to make sure they got shuffled correctly
#         idx = random.randint(0,len(learn.data.train_ds))
#         print(data.train_ds[idx][1], data.train_ds.items[idx])


    def on_batch_end(self, last_loss, last_output, last_target, train, **kwargs):
        
        if train:
            self.tloss_tot += last_loss
            self.tcount += 1
        else:
#             print('Entering on_batch_end for valididation.')
            self.vloss_tot += last_loss
            self.vcount += 1
            
            # Calculate accuracy
            
            output_ = last_output
#             print('initial output_: ', output_.shape)
            target = last_target
            target = target.cuda()
            target = target[0:-1:nview]
#             print('target:', target, target.shape)
            nsamp = int( output_.size(0) / nview )
#             print('nsamp: ', nsamp)
            batch_size = target.size(0)
#             print('batch_size: ', batch_size)

            num_classes = int(output_.size( 1 )/ nview) - 1
#             print('num_classes: ', num_classes)
            output_ = output_.view( -1, num_classes + 1 )
#             print('output_: ', output_)
            output_ = torch.nn.functional.log_softmax( output_, dim=1 ) #try sigmoid instead?
            output_ = output_[ :, :-1 ] - torch.t( output_[ :, -1 ].repeat(1, output_.size(1)-1).view(output_.size(1)-1, -1) )
            output_ = output_.view( -1, nview * nview, num_classes )
            output_ = output_.data.cpu().numpy()

            output_ = output_.transpose( 1, 2, 0 )
#             print('output_: ', output_.shape)
            scores = np.zeros( ( vcand.shape[ 0 ], num_classes, batch_size ) )
#             print('scores: ', scores.shape)
            output = torch.zeros( ( batch_size, num_classes ) )
    #         print('output: ', output)
    
            for j in range(vcand.shape[0]):
                for k in range(vcand.shape[1]):
                    scores[ j ] = scores[ j ] + output_[ vcand[ j ][ k ] * nview + k ]
#             print('scores: ', scores)

            for n in range( batch_size ):
                # For a given batch, n, np.argmax( scores[ :, :, n ] ) / scores.shape[ 1 ]
                # gets the index for the view that has the highest score, regardless of class.
                j_max = int( np.argmax( scores[ :, :, n ] ) / scores.shape[ 1 ] )
                self.viewOrd = np.append(self.viewOrd, j_max)
                # Scores[ j_max, :, n ] -- for batch n, view with highest score regardless of class.
                output[ n ] = torch.FloatTensor( scores[ j_max, :, n ] )
#                 print('scores[j_max,:,n]: ', scores[ j_max, :, n ], j_max)

            # For each batch, the scores for the views that had the highest score.
#             print('output:', output)
            output = output.cuda()
            # output[x,:] - view that had the highest class score for sample[x]
            # output[:,y] - score for class[y]

            topk = (1,5)
            maxk = max(topk)
            _, pred = output.topk(maxk, 1, True, True)
            pred = pred.t()
#             print('pred/target: ', pred, target)
#             _, pred40 = output.topk(40, 1, True, True)


            correct = pred.eq(target.contiguous().view(1, -1).expand_as(pred))

            prec = []
            for k in topk:
                correct_k = correct[:k].view(-1).float().sum(0)
                prec.append(correct_k.mul_(100.0 / batch_size))
                
            self.prec1 += prec[0]*(last_output.size(0)//nview)
            self.prec5 += prec[1]*(last_output.size(0)//nview)
            self.vcount_tot += (last_output.size(0)//nview)
            
            idx =  (correct[0] == 0).nonzero()
#             print(correct[0], idx)
            self.incTargs = np.append(self.incTargs, target[idx.view(-1)].data.cpu().numpy())
#             print(self.incTargs)
            
#             if prec[0] < 85.0:
# #                 print('prec1, prec5: ', prec)
# #                 print('\npred: ', pred[0])
#                 idx =  (correct[0] == 0).nonzero()
# #                 print(correct[0], idx.view(-1))
#                 print('Validation targets below 85%: ', target[idx.view(-1)], prec)

#             print('self.vcount_tot: ', self.vcount_tot)
            return {'last_output':output}


    def on_epoch_end(self, last_metrics, epoch, **kwargs):
        
        global viewOrd
        viewOrd = self.viewOrd.astype(int)
        
        print('\n*******EPOCH', epoch, '********\n')
        print('\nviewOrd: ', viewOrd, len(viewOrd))
        print('Error rate by class:')
        # Count classes that have been incorrectly classified
        u, counts = np.unique(self.incTargs, return_counts=True)
        v = [validCount[i] for i in u.astype(int)]
        unique = [classes[i] for i in u.astype(int)]
        corrects = ((counts/v * 100) + 0.5).astype(int) / 100.0
        countDict = dict(zip(unique, corrects))
        print(countDict, len(countDict))
        

#         print('vcount:', self.vcount)
        return add_metrics(last_metrics, [self.tloss_tot/self.tcount,
                                          self.prec1/self.vcount_tot, self.prec5/self.vcount_tot])

In [20]:
numClasses = (data.c+1)*nview
rotationNetHead = create_head(512, numClasses)

In [21]:
learn = cnn_learner(data, models.alexnet, metrics=None, pretrained=True,
                    custom_head = rotationNetHead, callback_fns=[GenerateTargetrN, rNAccuracy])
# learn = cnn_learner(data, models.resnet18, ps = 0.6, metrics=accuracy)

In [22]:
# learn.lr_find()
# learn.recorder.plot()

In [23]:
learn.fit_one_cycle(20, 1e-2)

epoch,train_loss,valid_loss,rN_t_loss,Prec@1,Prec@5,time


> /opt/anaconda3/lib/python3.7/site-packages/fastai/basic_train.py(26)loss_batch()
     24     breakpoint()
     25     #if not is_listy(yb): yb = [yb]
---> 26     out = model(*xb)
     27 
     28     # If training, update both out and target before finding loss.

ipdb> xb
[tensor([[[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          ...,
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],

         [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          ...,
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.428

ipdb> len(xb)
1
ipdb> xb[0]
tensor([[[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          ...,
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],

         [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          ...,
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286]],

         [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6

BdbQuit: 

In [26]:
# learn.save('stage_1')

In [32]:
learn.load('stage_1');

In [38]:
learn.unfreeze()

In [39]:
learn.fit_one_cycle(20, slice(1e-6, 1e-2))

epoch,train_loss,valid_loss,rN_t_loss,Prec@1,Prec@5,time
0,0.088160,17.557230,0.089666,90.964340,97.163696,02:45
1,0.083081,18.347692,0.082859,90.863045,97.163696,02:46
2,0.074197,19.106340,0.077210,90.869804,97.285255,02:45
3,0.074255,18.841602,0.072883,91.055511,97.325768,02:47
4,0.069703,19.650620,0.068665,91.264183,97.390602,02:46
5,0.065499,20.273008,0.064268,91.410049,97.406807,02:46
6,0.059964,20.106440,0.059495,91.548965,97.476265,02:46
7,0.054735,19.531006,0.054830,91.693680,97.503036,02:46
8,0.050273,21.293341,0.051032,91.842247,97.537361,02:47
9,0.047539,20.947386,0.047211,92.034042,97.593193,02:45



*******EPOCH 0 ********

Error rate by class:
{'bathtub': 0.14, 'bed': 0.01, 'bench': 0.25, 'bookshelf': 0.02, 'bottle': 0.01, 'chair': 0.01, 'cone': 0.1, 'cup': 0.35, 'curtain': 0.05, 'desk': 0.15, 'door': 0.1, 'dresser': 0.06, 'flower_pot': 0.85, 'glass_box': 0.04, 'keyboard': 0.05, 'lamp': 0.2, 'mantel': 0.09, 'monitor': 0.02, 'night_stand': 0.19, 'piano': 0.13, 'plant': 0.11, 'radio': 0.4, 'range_hood': 0.09, 'sink': 0.35, 'sofa': 0.02, 'stairs': 0.15, 'stool': 0.3, 'table': 0.14, 'tent': 0.1, 'tv_stand': 0.13, 'vase': 0.17, 'wardrobe': 0.5, 'xbox': 0.4} 33

*******EPOCH 1 ********

Error rate by class:
{'bathtub': 0.08, 'bed': 0.01, 'bench': 0.2, 'bookshelf': 0.03, 'bottle': 0.01, 'chair': 0.01, 'cone': 0.1, 'cup': 0.3, 'curtain': 0.05, 'desk': 0.19, 'dresser': 0.06, 'flower_pot': 0.85, 'glass_box': 0.04, 'keyboard': 0.05, 'lamp': 0.2, 'mantel': 0.09, 'monitor': 0.02, 'night_stand': 0.21, 'piano': 0.15, 'plant': 0.11, 'radio': 0.45, 'range_hood': 0.13, 'sink': 0.3, 'sofa': 0.02, 


*******EPOCH 15 ********

Error rate by class:
{'bathtub': 0.04, 'bench': 0.15, 'bookshelf': 0.05, 'bottle': 0.01, 'bowl': 0.1, 'cup': 0.2, 'curtain': 0.05, 'desk': 0.08, 'door': 0.05, 'flower_pot': 0.65, 'glass_box': 0.04, 'keyboard': 0.05, 'lamp': 0.15, 'mantel': 0.07, 'monitor': 0.02, 'night_stand': 0.08, 'piano': 0.06, 'plant': 0.07, 'radio': 0.25, 'range_hood': 0.06, 'sink': 0.1, 'sofa': 0.01, 'stairs': 0.05, 'stool': 0.15, 'table': 0.11, 'tent': 0.05, 'tv_stand': 0.04, 'vase': 0.1, 'wardrobe': 0.25, 'xbox': 0.2} 30

*******EPOCH 16 ********

Error rate by class:
{'bathtub': 0.04, 'bench': 0.15, 'bookshelf': 0.03, 'bottle': 0.01, 'bowl': 0.1, 'cup': 0.2, 'curtain': 0.05, 'desk': 0.07, 'door': 0.05, 'flower_pot': 0.55, 'glass_box': 0.04, 'keyboard': 0.05, 'lamp': 0.15, 'mantel': 0.08, 'monitor': 0.02, 'night_stand': 0.12, 'piano': 0.06, 'plant': 0.12, 'radio': 0.2, 'range_hood': 0.06, 'sink': 0.1, 'sofa': 0.01, 'stairs': 0.05, 'stool': 0.15, 'table': 0.09, 'tent': 0.05, 'tv_stand'

In [40]:
# learn.save('stage_2')

In [22]:
learn.load('stage_2');

## Predict class and pose

In [86]:
data.valid_ds.x.items[0:20]

array([PosixPath('ModelNet40_20/test/vase/vase_0539_001.png'), PosixPath('ModelNet40_20/test/vase/vase_0539_002.png'),
       PosixPath('ModelNet40_20/test/vase/vase_0539_003.png'), PosixPath('ModelNet40_20/test/vase/vase_0539_004.png'),
       PosixPath('ModelNet40_20/test/vase/vase_0539_005.png'), PosixPath('ModelNet40_20/test/vase/vase_0539_006.png'),
       PosixPath('ModelNet40_20/test/vase/vase_0539_007.png'), PosixPath('ModelNet40_20/test/vase/vase_0539_008.png'),
       PosixPath('ModelNet40_20/test/vase/vase_0539_009.png'), PosixPath('ModelNet40_20/test/vase/vase_0539_010.png'),
       PosixPath('ModelNet40_20/test/vase/vase_0539_011.png'), PosixPath('ModelNet40_20/test/vase/vase_0539_012.png'),
       PosixPath('ModelNet40_20/test/vase/vase_0539_013.png'), PosixPath('ModelNet40_20/test/vase/vase_0539_014.png'),
       PosixPath('ModelNet40_20/test/vase/vase_0539_015.png'), PosixPath('ModelNet40_20/test/vase/vase_0539_016.png'),
       PosixPath('ModelNet40_20/test/vase/vase_0

In [115]:
img = open_image(path/'test/night_stand/night_stand_0201_002.png')
batch = learn.data.one_item(img)
xb, yb = batch
img = open_image(path/'test/night_stand/night_stand_0201_019.png')
batch = learn.data.one_item(img)
xb1, yb = batch
xb = torch.cat((xb, xb1), dim=0)
xb = [xb]

In [116]:
out = learn.model(*xb)
batch_size = xb[0].size(0)
# print('batch_size: ', batch_size)
num_classes = int(out.size( 1 )/ nview) - 1
# print('num_classes: ', num_classes)
out = out.view( -1, num_classes + 1 )
out = torch.nn.functional.log_softmax( out, dim=1 )
out = out[ :, :-1 ] - torch.t( out[ :, -1 ].repeat(1, out.size(1)-1).view(out.size(1)-1, -1) )
for i in range(batch_size): 
    maxVals, idxs = torch.max(out[i*nview:i*nview+nview], dim=1)
    print('prediction:', classes[idxs[torch.argmax(maxVals)].item()], '-- pose:', torch.argmax(maxVals).item())
#     print('pose: ', torch.argmax(maxVals).item())

prediction: night_stand -- pose: 18
prediction: night_stand -- pose: 15


In [126]:
# Show validation views after calculating the "best view" for each group of 20 images
vimg = data.valid_ds.x.items
imgs = vimg[100:120]
vcand[viewOrd[5]]

fig=plt.figure(figsize=(30, 8))
columns = 10
rows = 2
j = 1
for i in vcand[viewOrd[4]]:
    img = PIL.Image.open(imgs[i])
    fig.add_subplot(rows, columns, j)
    j += 1
    plt.imshow(img)
plt.show()

array([ 3,  7,  2,  6,  1,  5,  0,  4, 13, 15, 12, 14, 18, 19, 16, 17, 11,  9, 10,  8])

In [36]:
learn.predict(img)

IndexError: list index out of range